In [1]:
import nb_setup 
import importlib
importlib.reload(nb_setup)
nb_setup.init()

Added project root to sys.path: /Users/falcon/Developments/act/backtesting-playground/trading
Changed current working directory to: /Users/falcon/Developments/act/backtesting-playground/trading
Initialized project with base directory: /Users/falcon/Developments/act/backtesting-playground/trading


In [2]:
import models.market as market
import importlib
importlib.reload(market)

market = market.Market.from_json_file()
df = market.to_df()
df.head()

,maxLeverage,name,onlyIsolated,szDecimals,dayNtlVlm,funding,impactPxs,markPx,midPx,openInterest,oraclePx,premium,prevDayPx
0,50,BTC,False,5,3.145381e+08,0.000013,"[64361.8, 64364.0]",64362.00000,64363.50000,1.737624e+03,64372.0000,-0.000124,63779.0000
1,50,ETH,False,4,2.403332e+08,0.000013,"[3139.77, 3139.9]",3139.90000,3139.85000,3.549263e+04,3138.6000,0.000373,3130.4000
2,20,ATOM,False,2,2.532351e+06,0.000013,"[8.2372, 8.2403]",8.23950,8.23980,1.036303e+05,8.2430,-0.000328,8.3849
3,20,MATIC,False,1,7.699034e+06,0.000013,"[0.71634, 0.71652]",0.71641,0.71651,2.689729e+06,0.7166,-0.000112,0.7003
4,20,DYDX,False,1,1.223493e+06,0.000013,"[2.18008, 2.18204]",2.18100,2.18155,6.026122e+05,2.1815,0.000000,2.1720


In [3]:
# get top n coins

def get_top_n_coins(n=60):
    top_n_coins_df = df.sort_values(by='dayNtlVlm', ascending=False).head(n)
    top_n_coins_df = top_n_coins_df.reset_index(drop=True)
    top_n_coins_df.head()

    # get first coin name
    top_n_coins = top_n_coins_df.get('name')
    first_coin = top_n_coins[0]
    print("top 1st coin:", first_coin)
    return top_n_coins

top_n_coins = get_top_n_coins(60)

top 1st coin: BTC


In [5]:
%time
import time
from hyperliquid.info import Info
from hyperliquid.utils import constants as hl_constants
import utils.now_timestamp as now_timestamp 
import utils.fetch_coin as fetch_coin
import models.candles as candles
import importlib
importlib.reload(fetch_coin)
importlib.reload(candles)
importlib.reload(now_timestamp)

info = Info(hl_constants.MAINNET_API_URL, skip_ws=True)
def fetch_coins_by_names(names):
    start = now_timestamp.get_timestamp_from_today_midnight_utc(days=-30)
    end = now_timestamp.get_timestamp_from_today_midnight_utc(days=0)
    error_coins = []
    for coin in names:
        try:
            time.sleep(0.2)
            df = fetch_coin.fetch_coin(
                coin,
                interval='4h',
                bar_count=380
            )
        except Exception as e:
            print(f"coin {coin}: not fetched")
            print(e)
            error_coins.append(coin)
            continue

fetch_coins_by_names(top_n_coins)
# fetch_coins_by_names(["BTC"])

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 6.2 µs
BTC
ETH
SOL
kPEPE
NEAR
WIF
kBONK
BNB
WLD
W
AVAX
FTM
TIA
ARB
DOGE
SEI
SUI
MATIC
OP
PENDLE
kSHIB
RNDR
BCH
AR
STX
INJ
APT
ENA
FET
ORDI
LTC
MKR
RUNE
HBAR
TON
CFX
ONDO
LDO
JTO
LINK
JUP
STRK
kFLOKI
ATOM
XRP
COMP
MAVIA
ADA
BOME
FIL
MERL
DOT
PYTH
GMX
APE
NEO
XAI
UNI
GALA
ENS
